# Intenssion:
ML-Models need numbers to work with. The webscraping does not provide enough data/features. We build some features by our own. 

1. built feature is the distance to the next train station
---
- **input**: address
- **output**: distance in meters


In [12]:
# Libraries
import os
import json
import urllib
import requests
import pandas as pd


In [23]:
address = "8400 Winterthur, Theaterstrasse 17.1"
#address = input("Enter the address: ")

In [24]:
# Define base url for address search
base_url= "https://api3.geo.admin.ch/rest/services/api/SearchServer?"

# Set up search parameters: address, origins and type
parameters = {"searchText": address,
              "origins": "address",
              "type": "locations",
             }

# Urllib.parse.urlencode turns parameters into url
print(f"{base_url}{urllib.parse.urlencode(parameters)}")

https://api3.geo.admin.ch/rest/services/api/SearchServer?searchText=8400+Winterthur%2C+Theaterstrasse+17.1&origins=address&type=locations


In [25]:
# Server request
r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")

# Get data in json-format
data = json.loads(r.content)
data
# can return several results, so we need to select the first one. It is the one closer to the input address. try: "8400 Winterthur, Theaterstrasse 17.1" vs "8400 Winterthur, Theaterstrasse 17"

{'results': [{'attrs': {'detail': 'theaterstrasse 17.1 8400 winterthur 230 winterthur ch zh',
    'featureId': '201032451_0',
    'geom_quadindex': '030010211112102121200',
    'geom_st_box2d': 'BOX(697167.118926371 262226.7588384855,697167.118926371 262226.7588384855)',
    'label': 'Theaterstrasse 17.1 <b>8400 Winterthur</b>',
    'lat': 47.50360870361328,
    'lon': 8.728364944458008,
    'num': 171,
    'objectclass': '',
    'origin': 'address',
    'rank': 7,
    'x': 262226.75,
    'y': 697167.125,
    'zoomlevel': 10},
   'id': 1553626,
   'weight': 4},
  {'attrs': {'detail': 'theaterstrasse 17 8400 winterthur 230 winterthur ch zh',
    'featureId': '2323240_0',
    'geom_quadindex': '030010211112013212300',
    'geom_st_box2d': 'BOX(697128.5668860576 262216.11449080077,697128.5668860576 262216.11449080077)',
    'label': 'Theaterstrasse 17 <b>8400 Winterthur</b>',
    'lat': 47.503517150878906,
    'lon': 8.727851867675781,
    'num': 17,
    'objectclass': '',
    'origin': '

In [26]:
data['results'][0]

{'attrs': {'detail': 'theaterstrasse 17.1 8400 winterthur 230 winterthur ch zh',
  'featureId': '201032451_0',
  'geom_quadindex': '030010211112102121200',
  'geom_st_box2d': 'BOX(697167.118926371 262226.7588384855,697167.118926371 262226.7588384855)',
  'label': 'Theaterstrasse 17.1 <b>8400 Winterthur</b>',
  'lat': 47.50360870361328,
  'lon': 8.728364944458008,
  'num': 171,
  'objectclass': '',
  'origin': 'address',
  'rank': 7,
  'x': 262226.75,
  'y': 697167.125,
  'zoomlevel': 10},
 'id': 1553626,
 'weight': 4}

In [27]:
Latitude = data['results'][0]['attrs']['lat']
Latitude

47.50360870361328

In [29]:
Longitude = data['results'][0]['attrs']['lon']
Longitude

8.728364944458008